# Part 1

In [1]:
import numpy as np
from io import StringIO

In [2]:
test_input = """..F7.
.FJ|.
SJ.L7
|F--J
LJ...
"""

In [3]:
pipe_map = {
    '|':('║', '│'),
    '-': ('═', '─'),
    'L': ('╚', '└'),
    'J':('╝', '┘'),
    '7':('╗', '┐'),
    'F': ('╔', '┌'),
}

In [4]:
def print_pipes(input):
    for k, v in pipe_map.items():
        input = input.replace(k, v[1])
    print(input)

In [5]:
print_pipes(test_input)

..┌┐.
.┌┘│.
S┘.└┐
│┌──┘
└┘...



In [6]:
def get_grid(fn):
    lines = np.genfromtxt(fn, dtype=str)
    grid = np.array([np.array(list(l)) for l in lines])
    return np.pad(grid, pad_width=1, constant_values=".")

In [9]:
pathways = {
    (-1,0): ('F', '7', '|'), # Up
    (1,0): ('L', 'J', '|'),  # Down
    (0,-1): ('J', '7', '-'), # Left
    (0,1): ('F', 'L', '-'),  # Right
}

directions = {
    '|': ((1,0),(-1,0)),
    '-': ((0,-1),(0,1)),
    'F': ((1,0),(0,1)),
    '7': ((1,0),(0,-1)),
    'J': ((-1,0),(0,-1)),
    'L': ((-1,0),(0,1)),
}

In [14]:
def get_path(grid):
    # Find the start
    path = [np.argwhere(grid=='S')[0]]

    # First step
    for dir, valid in pathways.items():
        _idx = path[-1] + dir
        if grid[*_idx] in valid:
            path.append(_idx)
            break
        
    # Rest
    while True:
        # Next
        pipe = grid[*path[-1]]
        if pipe == 'S':
            break

        connected_cells = path[-1] + directions[pipe]
        next_cell = connected_cells[~np.apply_along_axis(all, 1, connected_cells == path[-2])][0]
        path.append(next_cell)
    return path

In [15]:
def part1(input):

    grid = get_grid(input)

    path = get_path(grid)

    print(int(np.floor(len(path)/2)))

In [16]:
part1(StringIO(test_input))

8


In [17]:
part1("inputs/day10.txt")

6903


# Part 2

In [39]:
border_map = {
    ("|", (-1,0)): {(0, 1): "A", (0,-1): "B"},
    ("|", (1,0)): {(0, 1): "B", (0,-1): "A"},
    ("-", (0,1)): {(-1,0): "B", (1, 0): "A"},
    ("-", (0,-1)): {(-1,0): "A", (1, 0): "B"},
    ("L", (-1,0)): {(1, 0): "B", (0,-1): "B"},
    ("L", (0,1)): {(1, 0): "A", (0,-1): "A"},
    ("J", (0,-1)): {(1, 0): "B", (0, 1): "B"},
    ("J", (-1,0)): {(1, 0): "A", (0, 1): "A"},
    ("7", (0,-1)): {(-1,0): "A", (0, 1): "A"},
    ("7", (1,0)): {(-1,0): "B", (0, 1): "B"},
    ("F", (0,1)): {(-1,0): "B", (0,-1): "B"},
    ("F", (1,0)): {(-1,0): "A", (0,-1): "A"},
}

In [38]:
def get_first_fills(grid, path):

    grid_fills = grid.copy()
    _path = [(p[0], p[1]) for p in path]
    for i in range(1,len(path)-1):
        _y, _x = path[i-1]-path[i]
        _p = grid[*path[i]]
        border_map[(_p, (_y, _x))]
        
        for k, v in border_map[(_p, (_y, _x))].items():
            _cell = path[i] + k
            if not tuple(_cell) in _path:
                grid_fills[*_cell] = v

    return grid_fills

In [42]:
def _fill_grid(grid, path):
    """Fill the grid with A's and B's, """
    _path = [(p[0], p[1]) for p in path]
    for idx, x in np.ndenumerate(grid):
        # if in path continue 
        if idx in _path:
            continue
        if x in ('A', 'B'):
            continue

        for dir in np.array([[0,1], [0,-1], [1,0], [-1,0]]):
            try:
                if grid[*(idx + dir)] == 'A':
                    grid[idx] = 'A'
                elif grid[*(idx + dir)] == 'B':
                    grid[idx] = 'B'
            except:
                pass
    return grid


def fill_grid(grid, path):
    
    while True:
        n_filled = ((grid == 'A') + (grid == 'B')).sum()
        grid = _fill_grid(grid, path)
        
        if ((grid == 'A') + (grid == 'B')).sum() == n_filled:
            break
    return grid

In [43]:
def print_grid(grid, symbols=True):
    print_pipes("\n".join(["".join(g) for g in grid]), symbols)

In [44]:
def print_pipes(input, symbols=True):
    if symbols:
        for k, v in pipe_map.items():
            input = input.replace(k, v[1])
    print(input)

In [45]:
def part2(input):
    grid = get_grid(input)
    path = get_path(grid)

    part_filled_grid = get_first_fills(grid, path)
    grid = fill_grid(part_filled_grid, path)

    print('A', (grid == 'A').sum())
    print('B', (grid == 'B').sum())
    print_grid(grid)
    

In [46]:
part2("inputs/day10.txt")

A 6093
B 265
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA┌┐AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA┌──┐AAAAAAAAAAAAA│└─┐AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA┌─┐AAAAAAAAAAAA│┌─┘AAAAAAAAA┌┐AA└─┐└┐AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA